# TensorBoard Debugger notebook
---

<font color='red'> <h3>Tested with TensorFlow 1.7</h3></font>

This notebook introduces how to connect your TensorFlow application the TensorBoard debugger.

In [ ]:
def wrapper():
    # Copyright 2016 The TensorFlow Authors. All Rights Reserved.
    #
    # Licensed under the Apache License, Version 2.0 (the "License");
    # you may not use this file except in compliance with the License.
    # You may obtain a copy of the License at
    #
    #     http://www.apache.org/licenses/LICENSE-2.0
    #
    # Unless required by applicable law or agreed to in writing, software
    # distributed under the License is distributed on an "AS IS" BASIS,
    # WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    # See the License for the specific language governing permissions and
    # limitations under the License.
    # ==============================================================================
    """Demo of the tfdbg curses CLI: Locating the source of bad numerical values.

    The neural network in this demo is larged based on the tutorial at:
      tensorflow/examples/tutorials/mnist/mnist_with_summaries.py

    But modifications are made so that problematic numerical values (infs and nans)
    appear in nodes of the graph during training.
    """

    import argparse
    import sys
    import os

    import tensorflow as tf

    from tensorflow.examples.tutorials.mnist import input_data
    from tensorflow.python import debug as tf_debug
    
    from hops import tensorboard

    IMAGE_SIZE = 28
    HIDDEN_SIZE = 500
    NUM_LABELS = 10
    RAND_SEED = 42
    LEARNING_RATE = 0.025
    STEPS = 10
    
    tensorboard_debug_address = tensorboard.interactive_debugger()
    
    # How you easily get the TensorBoard logdir for summaries
    tensorboard_logdir = tensorboard.logdir()

    # Import data
    mnist = input_data.read_data_sets(os.getcwd(),
                                        one_hot=True,
                                        fake_data=False)

    def feed_dict(train):
        if train:
            xs, ys = mnist.train.images, mnist.train.labels
        else: 
            xs, ys = mnist.test.images, mnist.test.labels
            
        return {x: xs, y_: ys}

    sess = tf.InteractiveSession()

    # Create the MNIST neural network graph.

    # Input placeholders.
    with tf.name_scope("input"):
      x = tf.placeholder(
          tf.float32, [None, IMAGE_SIZE * IMAGE_SIZE], name="x-input")
      y_ = tf.placeholder(tf.float32, [None, NUM_LABELS], name="y-input")

    def weight_variable(shape):
      """Create a weight variable with appropriate initialization."""
      initial = tf.truncated_normal(shape, stddev=0.1, seed=RAND_SEED)
      return tf.Variable(initial)

    def bias_variable(shape):
      """Create a bias variable with appropriate initialization."""
      initial = tf.constant(0.1, shape=shape)
      return tf.Variable(initial)

    def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
      """Reusable code for making a simple neural net layer."""
      # Adding a name scope ensures logical grouping of the layers in the graph.
      with tf.name_scope(layer_name):
        # This Variable will hold the state of the weights for the layer
        with tf.name_scope("weights"):
          weights = weight_variable([input_dim, output_dim])
        with tf.name_scope("biases"):
          biases = bias_variable([output_dim])
        with tf.name_scope("Wx_plus_b"):
          preactivate = tf.matmul(input_tensor, weights) + biases

        activations = act(preactivate)
        return activations

    hidden = nn_layer(x, IMAGE_SIZE**2, HIDDEN_SIZE, "hidden")
    logits = nn_layer(hidden, HIDDEN_SIZE, NUM_LABELS, "output", tf.identity)
    y = tf.nn.softmax(logits)

    with tf.name_scope("cross_entropy"):

      diff = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits)
      with tf.name_scope("total"):
        cross_entropy = tf.reduce_mean(diff)

    with tf.name_scope("train"):
      train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(
          cross_entropy)

    with tf.name_scope("accuracy"):
      with tf.name_scope("correct_prediction"):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
      with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    sess.run(tf.global_variables_initializer())
    sess = tf_debug.TensorBoardDebugWrapperSession(
            sess, tensorboard_debug_address)

    # Add this point, sess is a debug wrapper around the actual Session if
    for i in range(STEPS):
      
      acc = sess.run(accuracy, feed_dict=feed_dict(False))
      print("Accuracy at step %d: %s" % (i, acc))

      sess.run(train_step, feed_dict=feed_dict(True))




In [ ]:
from hops import experiment
experiment.launch(spark, wrapper)

## Getting to TensorBoard <a class="anchor" id='tensorboard'></a>
To find the TensorBoard for the execution, please go back to HopsWorks and follow the arrows in the images below.

![Image7-Monitor.png](../../images/jupyter.png)
![Image4-LaunchTensorboard.png](../../images/overview.png)
![Image8-Tensorboard.png](../../images/tensorboard_debug.png)